In [3]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
from tqdm import tqdm

#use for naive_bayes
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.tokenize import TweetTokenizer
import pdb
from nltk.corpus import twitter_samples
import numpy as np
import pandas as pd
import nltk
from os import getcwd

from tensorflow.keras import layers
from tensorflow.keras import losses

nltk.download('stopwords')

#use for LSTM and WordEmbedding Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from  nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#https://www.tensorflow.org/text/tutorials/fine_tune_bert

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
data_path = '/content/drive/MyDrive/Big_Data/custom_model/clean_data.csv'
data = pd.read_csv(data_path,delimiter = ',',encoding = "ISO-8859-1")

data["text"] = str(data["text"])
convert_dict =  {0: "NEGATIVE", 2: "NEUTRAL", 4: "POSITIVE"}
data["target"].replace(convert_dict,inplace=True)

In [54]:
train_df, remaining = train_test_split(data, random_state=42, train_size=0.8, stratify=data.target.values)
valid_df, test_df = train_test_split(remaining, random_state=42, train_size=0.08, stratify=remaining.target.values)
train_df.shape, valid_df.shape, test_df.shape

((1280000, 6), (25600, 6), (294400, 6))

In [55]:
train_data = tf.data.Dataset.from_tensor_slices((train_df.text.values, train_df.target.values))
valid_data = tf.data.Dataset.from_tensor_slices((valid_df.text.values, valid_df.target.values))
test_data = tf.data.Dataset.from_tensor_slices((test_df.text.values, test_df.target.values))

for text, label in train_data.take(1):
    print(text)
    print(label)

tf.Tensor(b"0                                                        NaN\n1          upset can't updat facebook text ... might cri ...\n2            dive mani time ball manag save 50 rest go bound\n3                            whole bodi feel itchi like fire\n4                                    behav i'm mad can't see\n                                 ...                        \n1599995                           woke school best feel ever\n1599996     thewdb.com cool hear old walt interview \xc3\x83\xc2\xa2 \xc3\x82\xc2\x99 \xc3\x82\xc2\xab\n1599997                         readi mojo makeov ask detail\n1599998    happi 38th birthday boo alll time tupac amaru ...\n1599999                                 happi charitytuesday\nName: text, Length: 1600000, dtype: object", shape=(), dtype=string)
tf.Tensor(b'POSITIVE', shape=(), dtype=string)


#BERT MODEL

In [14]:
! git clone --depth 1 -b v2.8.0 https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [19]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import sys
sys.path.append('models')
from official.nlp.data import classifier_data_lib
from official.nlp.bert import tokenization
from official.nlp import optimization

In [20]:
print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TF Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [21]:
label_list = [0, 1] # Label categories
max_seq_length = 300 # maximum length of (token) input sequences
train_batch_size = 1024

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2",
                            trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [22]:
print(tokenizer.wordpiece_tokenizer.tokenize('hi, how are you doing?'))
print(tokenizer.convert_tokens_to_ids(tokenizer.wordpiece_tokenizer.tokenize('hi, how are you doing?')))

['hi', '##,', 'how', 'are', 'you', 'doing', '##?']
[7632, 29623, 2129, 2024, 2017, 2725, 29632]


In [23]:
def to_feature(text, label, label_list=label_list, max_seq_length=max_seq_length, tokenizer=tokenizer):
    example = classifier_data_lib.InputExample(guid = None,
                                              text_a = text.numpy(), 
                                              text_b = None, 
                                              label = label.numpy())
    feature = classifier_data_lib.convert_single_example(0, example, label_list,
                                      max_seq_length, tokenizer)
    
    return (feature.input_ids, feature.input_mask, feature.segment_ids, feature.label_id)

In [25]:
def to_feature_map(text, label):
    input_ids, input_mask, segment_ids, label_id = tf.py_function(to_feature, inp=[text, label], 
                                  Tout=[tf.int32, tf.int32, tf.int32, tf.int32])

    # py_func doesn't set the shape of the returned tensors.
    input_ids.set_shape([max_seq_length])
    input_mask.set_shape([max_seq_length])
    segment_ids.set_shape([max_seq_length])
    label_id.set_shape([])

    x = {
          'input_word_ids': input_ids,
          'input_mask': input_mask,
          'input_type_ids': segment_ids
      }
    return (x, label_id)

In [56]:
train_data = (train_data.map(to_feature_map,
                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
                          #.cache()
                          .shuffle(1000)
                          .batch(32, drop_remainder=True)
                          .prefetch(tf.data.experimental.AUTOTUNE))
valid_data = (valid_data.map(to_feature_map,
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)
                          .batch(32, drop_remainder=True)
                          .prefetch(tf.data.experimental.AUTOTUNE))
test_data = (test_data.map(to_feature_map,
                            num_parallel_calls=tf.data.experimental.AUTOTUNE)
                          .batch(32, drop_remainder=True)
                          .prefetch(tf.data.experimental.AUTOTUNE))

In [47]:
train_data.element_spec

({'input_mask': TensorSpec(shape=(32, 300), dtype=tf.int32, name=None),
  'input_type_ids': TensorSpec(shape=(32, 300), dtype=tf.int32, name=None),
  'input_word_ids': TensorSpec(shape=(32, 300), dtype=tf.int32, name=None)},
 TensorSpec(shape=(32,), dtype=tf.int32, name=None))

In [61]:
def create_model():
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                        name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="input_mask")
    input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="input_type_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])

    drop = tf.keras.layers.Dropout(0.4)(pooled_output)
    output = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(drop)

    model = tf.keras.Model(
      inputs={
          'input_word_ids': input_word_ids,
          'input_mask': input_mask,
          'input_type_ids': input_type_ids
      },
      outputs=output)
    return model  

In [62]:
model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 300)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 300)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 300)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 300, 768)]                'input_mask[0][0]',       

In [63]:
epochs = 20
history = model.fit(train_data,
                    validation_data=valid_data,
                    epochs=epochs,
                    verbose=1)

Epoch 1/20


UnknownError: ignored

In [59]:
data["target"].value_counts()

NEGATIVE    800000
POSITIVE    800000
Name: target, dtype: int64